In [32]:
# automatically load changes from the sandbox_steps module
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import sys
import os
from e2b_desktop import Sandbox, CommandExitException
from dotenv import load_dotenv
load_dotenv()

sys.path.append(os.path.abspath('..')) # Add the parent directory (project root) to sys.path

from counter_strike.install_cs import install_cs_1_6
from counter_strike.image_handling import get_screenshot_message, draw_point, get_screenshot

from llms.models import AimingModel

E2B_API_KEY = os.environ.get("E2B_API_KEY")
CS_SERVER_IP = os.environ.get("CS_SERVER_IP")


In [34]:
# With custom configuration
desktop = Sandbox(
    display=":0",  # Custom display (defaults to :0)
    resolution=(1920, 1080),  # Custom resolution
    timeout = 3600) 

desktop.stream.start()

# Get stream URL
url = desktop.stream.get_url()
print(url)

# only viewing 
url_view = desktop.stream.get_url(view_only=True)
print(url_view)

https://6080-imh51qd8ks5os9p4e30i4-f3c1fb85.e2b.app/vnc.html?autoconnect=true&resize=scale
https://6080-imh51qd8ks5os9p4e30i4-f3c1fb85.e2b.app/vnc.html?autoconnect=true&view_only=true&resize=scale


In [35]:
install_cs_1_6(desktop=desktop)

### Steps to connect to the server

In [ ]:
def connect_to_server(desktop: "Sandbox"):
    desktop.wait(5000)
    desktop.left_click(76, 948) # click on Find Servers
    desktop.wait(1000)
    desktop.left_click(258, 76) # Click on Favorites Tab
    desktop.wait(1000)
    desktop.left_click(665, 604) # Add server
    desktop.wait(1000)
    desktop.write(text=CS_SERVER_IP) # Type server ip Address
    desktop.wait(500)
    desktop.left_click(1119, 527) # Add server to favourites
    desktop.wait(500)
    desktop.left_click(273, 126) # Select the added server
    desktop.wait(500)
    desktop.left_click(849, 606) # Connect
    # Now wait at least 2 mins for connection and Download
    for i in range(5):
        print(i)
        desktop.wait(30_000)

connect_to_server(desktop)

0
1


In [ ]:
# steps to connect to the game
desktop.press("enter")
desktop.wait(300)
desktop.press("1") # 1=T, 2=CT, 6=SPECTATE
desktop.wait(300)
desktop.press("4") # T side: Guerilla warfare skin because they have the red headband

# congrats, you joined the game!!

In [ ]:
desktop.left_click(170, 10)     

In [3]:
desktop.kill()

In [11]:
from counter_strike.image_handling import get_mouse_movements
from counter_strike.controls import aim, shoot

from llms.models import AimingModel

aiming_model = AimingModel(model="qwen/qwen2.5-vl-72b-instruct")

['qwen/qwen2.5-vl-32b-instruct', 'qwen/qwen2.5-vl-72b-instruct', 'qwen/qwen-2.5-vl-7b-instruct', 'qwen/qwen2.5-vl-32b-instruct:free']
qwen/qwen2.5-vl-72b-instruct
['qwen/qwen2.5-vl-32b-instruct', 'qwen/qwen-2.5-vl-7b-instruct', 'qwen/qwen2.5-vl-32b-instruct:free']


In [12]:
from llms.tools import MoveTool
from llms.models import OpenRouterGameplayModel

move_tool = MoveTool(desktop=desktop)
tools_move = {move_tool.name: move_tool}

gameplay_model = OpenRouterGameplayModel(tools=tools_move, model="google/gemini-2.5-flash-preview")
move_tool.name

'move_tool'

In [96]:
user_message = [{
    "role": "user",
    "content": [
        {
            "type": "text",
            "text": f"This is the current state of your counterstrike deathmatch game. Choose the keystrokes to do and execute them with the move tool. It's up to you to choose the best action.",
        },
        ]
    }
]

messages = screenshot_message + user_message

In [97]:
gameplay_model.complete(messages)

Arguments: {'key_sequence': 'wwwwwwwwwwwwwwwwwwww'}
The model halucinated a tool move_character_directional. The tool is not defined.


('',
 ChatCompletion(id='gen-1746872604-PaJ4DjO3AC8NJGYpOG08', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='tool_0_move_character_directional', function=Function(arguments='{"key_sequence":"wwwwwwwwwwwwwwwwwwww"}', name='move_character_directional'), type='function', index=0)], reasoning=None), native_finish_reason='STOP')], created=1746872604, model='google/gemini-2.5-flash-preview', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=11, prompt_tokens=1941, total_tokens=1952, completion_tokens_details=None, prompt_tokens_details=None), provider='Google'))

In [31]:
for i in range(3):
    user_message = [{
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": f"This is the current state of your counterstrike deathmatch game. Choose the keystrokes to do and execute them with the move tool. It's up to you to choose the best action.",
            },
            ]
        }
    ]

    print(f"Iteration {None}")
    screenshot_message = get_screenshot_message(desktop, filename="../images/screenshot.jpg")

    point_json, _ = aiming_model.complete(user_messages=screenshot_message, debug=True)
    print(point_json)
    coords = aiming_model.parse_point_json(point_json)
    print(f"coords: {coords}")
    if not coords:
        # TODO: Define agentic logic to move around 
        print("Letting gemini move:")
        messages = screenshot_message + user_message
        print(gameplay_model.complete(messages)) # will use tools to move around
        pass

    else:
        draw_point(point = coords, image_path="../images/screenshot.jpg", output_path="../images/screenshot_annotated.jpg")

        mouse_movements = get_mouse_movements(coords=coords)
        aim(mouse_movements, desktop=desktop)
        shoot(desktop=desktop)

Iteration None
ChatCompletion(id='gen-1746874166-aClgoP9lS2FziF4t1EQ6', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"point": {"x": "80", "y": "440"}}', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning=None), native_finish_reason='stop')], created=1746874166, model='qwen/qwen2.5-vl-72b-instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=19, prompt_tokens=2790, total_tokens=2809, completion_tokens_details=None, prompt_tokens_details=None), provider='Parasail')
{"point": {"x": "80", "y": "440"}}
coords: {'x': 80, 'y': 440}
Aiming to coords 1: {'x': 0, 'y': 410}
Aiming to coords 2: {'x': 776, 'y': 540}
------------------------------
Iteration None
ChatCompletion(id='gen-1746874178-n5Ndu7wqB1162Sxn3uTD', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"p